In [2]:
import os
import base64
from io import BytesIO
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [16]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [17]:

def identify_product_from_image(image_path_or_url):
    """
    Identifies a product from an image using LangChain with Gemini.

    Args:
        image_path_or_url: The local file path or URL of the product image.

    Returns:
        A string containing the identified product information or an error message.
    """
    try:
        # --- 1. Load Image ---
        if image_path_or_url.startswith(("http://", "https://")):
            # Handle URL
            response = requests.get(image_path_or_url)
            response.raise_for_status()
            image_bytes = response.content
        else:
            # Handle local file path
            with open(image_path_or_url, "rb") as image_file:
                image_bytes = image_file.read()

        # --- 2. Prepare the Prompt and Image for LangChain ---
        prompt_text = """
        Analyze the provided image of a product and return the following information in a structured manner with clear labels without bold or italics. The information to extract includes:
        1.  Product Name: The primary name of the product.
        2.  Product Type: The category of the product (e.g., Food, Cosmetic, Cleaning Supply).
        3.  Extracted Information: All ingredients and chemicals, and warnings and certifications.
        4.  Brand: The brand name of the product.
        Provide the response in a clear and concise format. If any information is not available, indicate it as "Not Available". Do not include any additional commentary or explanations.
        """
        
        image_b64 = base64.b64encode(image_bytes).decode("utf-8")

        message = HumanMessage(
            content=[
                {"type": "text", "text": prompt_text},
                {
                    "type": "image_url",
                    "image_url": f"data:image/jpeg;base64,{image_b64}",
                },
            ]
        )

        # --- 3. Call the Gemini API via LangChain ---
        response = llm.invoke([message])

        # --- 4. Process and Return the Response ---
        if response and response.content:
            return response.content
        else:
            return "Failed to get a valid response from the Gemini API."

    except FileNotFoundError:
        return f"Error: Image file not found at: {image_path_or_url}"
    except requests.exceptions.RequestException as e:
        return f"Error: Failed to fetch image from URL: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

# --- Example Usage ---

if __name__ == "__main__":
    # --- Get Image from User ---
    print("Please provide the path to the product image.")
    image_location = input("Enter local file path or image URL: ").strip()

    if not image_location:
        print("No image location provided. Exiting.")
    else:
        # --- Identify Product ---
        print("\nAnalyzing product... Please wait.")
        product_info = identify_product_from_image(image_location)

        # --- Display Results ---
        print("\n--- Product Identification Results ---")
        print(product_info)
        print("\n------------------------------------")


Please provide the path to the product image.

Analyzing product... Please wait.

--- Product Identification Results ---
Product Name: Kurkure
Product Type: Food
Extracted Information:
Ingredients: Cereal Products (57%) (Rice Meal (38%), Corn Meal (19%)), Edible Vegetable Oil (Palmolein), Seasoning (Cinnamon, Spices & Condiments, Sugar, Iodised Salt, Acidity Regulators (330, 296), Flavour (Natural and Nature Identical Flavouring Substances), Flavour Enhancers (627, 631), Edible Vegetable Oil (Corn Oil), Color (160c), Emulsifier (414)), Gram Meal (6%).
Chemicals: Acidity Regulators (330, 296), Flavour Enhancers (627, 631), Color (160c), Emulsifier (414).
Warnings: As flavouring agent. Contains Onion and Garlic. Pack less than 20 g is a single consumption pack.
Certifications: FSSAI (Mkt. Lic. No. 10014064000435), Vegetarian symbol (green square with circle).
Brand: PepsiCo, Inc.

------------------------------------


In [19]:
product_info

'Product Name: Kurkure\nProduct Type: Food\nExtracted Information:\nIngredients: Cereal Products (57%) (Rice Meal (38%), Corn Meal (19%)), Edible Vegetable Oil (Palmolein), Seasoning (Cinnamon, Spices & Condiments, Sugar, Iodised Salt, Acidity Regulators (330, 296), Flavour (Natural and Nature Identical Flavouring Substances), Flavour Enhancers (627, 631), Edible Vegetable Oil (Corn Oil), Color (160c), Emulsifier (414)), Gram Meal (6%).\nChemicals: Acidity Regulators (330, 296), Flavour Enhancers (627, 631), Color (160c), Emulsifier (414).\nWarnings: As flavouring agent. Contains Onion and Garlic. Pack less than 20 g is a single consumption pack.\nCertifications: FSSAI (Mkt. Lic. No. 10014064000435), Vegetarian symbol (green square with circle).\nBrand: PepsiCo, Inc.'

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

text = f"Can you provide information about {product_info}? Please include details such as its benefits, and any safety considerations. and also include whether the product or the ingredients are banned or allowed in various countries. Provide information in a structured and concise format with clear labels without bold or italics."

chat_history = [
    SystemMessage(content='You are an expert in human health and dietary supplements.'),
    HumanMessage(content=f'{text}')
]

result = llm.invoke(chat_history)
chat_history.append(AIMessage(content=result.content))
print("\n--- Detailed Product Information ---")
print(result.content)




--- Detailed Product Information ---
Product Information
Product name: Kurkure
Product type: Food (processed snack)
Brand: PepsiCo, Inc.

Ingredients Overview
Kurkure is primarily composed of cereal products (rice meal, corn meal) and gram meal (chickpea flour). It is cooked in edible vegetable oil (palmolein). The product contains a seasoning blend with spices, sugar, iodised salt, and various food additives.
Key ingredients: Rice meal, corn meal, palmolein oil, gram meal.
Seasoning components: Cinnamon, other spices and condiments, sugar, iodised salt, corn oil.
Chemicals (Food Additives):
 Acidity Regulators: 330 (Citric Acid), 296 (Malic Acid) – used for tartness and flavor.
 Flavour Enhancers: 627 (Disodium Guanylate), 631 (Disodium Inosinate) – enhance savory flavors.
 Color: 160c (Paprika extract) – provides orange-red color.
 Emulsifier: 414 (Acacia Gum) – helps stabilize ingredients.

Potential Benefits
Kurkure is a processed snack and offers minimal nutritional benefits.
Ene